In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load data

In [ ]:
df= pd.read_csv('../input/leaf-classification/train.csv.zip', index_col='id')
df

In [ ]:
for col in df.columns:
    if df[col].isna().sum() > 0:
        print(col, df[col].isna().sum()   /len(df))

> There's no missing value 

In [ ]:
df.species.value_counts()

In [ ]:
len(df.species.unique())

In [ ]:
y=df.species
y.head()

In [ ]:
X = df.drop(columns = 'species', axis=1)
X.head()

# Label Encoding 

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder= LabelEncoder().fit(y)
labeled_y= label_encoder.transform(y)


In [ ]:
classes=list(label_encoder.classes_)
classes

# Define Parameters

In [ ]:
parameters = {
    'n_estimators': list(range(300, 501, 100)), 
    'max_leaf_nodes': list(range(10, 30, 5)), 
    'max_depth': list(range(10, 40, 3))
}
print(parameters)

# Define GridSearchCV for hypter-parameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import log_loss

gsearch = GridSearchCV(estimator=XGBClassifier(),
                       param_grid = parameters, 
                       scoring= 'neg_log_loss',
                       n_jobs=4,cv=5, verbose=7)

# Fit the model 

In [ ]:
gsearch.fit(X,labeled_y)

# Get the best parameters

In [ ]:
best_n_estimators = gsearch.best_params_.get('n_estimators')
best_n_estimators

In [ ]:
best_max_leaf_nodes = gsearch.best_params_.get('max_leaf_nodes')
best_max_leaf_nodes

In [ ]:
best_max_depth = gsearch.best_params_.get('max_depth')
best_max_depth

# Build and fit the final model with the best parameters

In [ ]:
final_model = XGBClassifier(n_estimators=best_n_estimators,
                            max_leaf_nodes=best_max_leaf_nodes,
                            max_depth=best_max_depth)

In [ ]:
final_model.fit(X, labeled_y)

In [ ]:
test = pd.read_csv('../input/leaf-classification/test.csv.zip',index_col='id')

# Predict the test data

In [ ]:
pred_test=final_model.predict_proba(test)
pred_test.shape

In [ ]:
pred_test

In [ ]:
output = pd.DataFrame(pred_test, columns=classes)
output.insert(0, 'id', test.index)
output.reset_index()

output.to_csv('submission.csv', index=False)
print('done')
output